# Логистическая регрессия. Практическая работа

## Цель практической работы

Вы уже делали задание, в котором с помощью метода ближайших соседей оценивали склонность клиента банка откликнуться или не откликнуться на предложение.

В этом практическом задании ваши цели:
*  решить эту же задачу с помощью логистической регрессии;
*  потренироваться в подборе порога; 
*  потренироваться в подборе гиперпараметров модели.

## Что входит в работу

*  Загрузить данные для задачи.
*  Обучить метод ближайших соседей с заданным количеством соседей k, вычислить метрики.
*  Обучить логистическую регрессию с параметрами по умолчанию, вычислить метрики.
*  Подобрать порог модели, вычислить метрики.
*  Подобрать гиперпараметр С (константа регуляризации) модели, вычислить метрики.

## Что оценивается

*  Выполнены все этапы задания: код запускается, отрабатывает без ошибок; подробно и обоснованно написаны текстовые выводы, где это требуется.

## Формат сдачи
Выполните предложенные задания — впишите свой код (или, если требуется, текст) в ячейки после комментариев. 

*Комментарии — это текст, который начинается с символа #. Например: # ваш код здесь.*

Сохраните изменения, используя опцию Save and Checkpoint из вкладки меню File или кнопку Save and Checkpoint на панели инструментов. Итоговый файл в формате .ipynb (файл Jupyter Notebook) загрузите в личный кабинет и отправьте на проверку.

In [44]:
# подключим библиотеки
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [45]:
# считаем данные
data = pd.read_csv('ClientsData.csv')

In [46]:
data.head()

,AGE,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,GENDER,CHILD_TOTAL,DEPENDANTS,PERSONAL_INCOME,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED,LOAN_DLQ_NUM,TARGET
0,49,1,0,1,2,1,5000.0,1,1,2,0
1,32,1,0,1,3,3,12000.0,1,1,1,0
2,52,1,0,1,4,0,9000.0,2,1,0,0
3,39,1,0,1,1,1,25000.0,1,1,3,0
4,30,1,0,0,0,0,12000.0,2,1,2,0


В этом ноутбуке нам придётся подбирать гиперпараметры модели, а ещё порог. Поэтому, чтобы не переобучиться, разобъём данные на трейн, валидацию и тест.

*  Обучать модели будем на тренировочных данных.
*  Подбирать необходимые величины — по валидации.
*  Оценивать качество — на тесте.

In [47]:
# разделим данные на обучающую и тестовую выборки
from sklearn.model_selection import train_test_split

X = data.drop('TARGET', axis=1)
y = data['TARGET']

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=0.7, random_state=123)
Xtrain, Xval, ytrain, yval = train_test_split(Xtrain, ytrain, train_size=0.7, random_state=123)

В задании по методу ближайших соседей было найдено, что оптимальное число соседей k = 9.

Обучите на тренировочных данных KNN с k = 9 и выведите матрицу ошибок, а также значение метрик precision и recall на тестовых данных.

In [48]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score

In [49]:
k = 9
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(Xtrain, ytrain)

KNeighborsClassifier(n_neighbors=9)

In [50]:
ypred = knn.predict(Xtest)

conf_matrix = confusion_matrix(ytest, ypred)
precision = precision_score(ytest, ypred)
recall = recall_score(ytest, ypred)

print("Матрица ошибок:")
print(conf_matrix)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")

Матрица ошибок:
[[3982   26]
 [ 553    6]]
Precision: 0.19
Recall: 0.01


Какой вывод можно сделать:
- для класса 0 — клиент не откликнулся — мы получили достаточно высокие значения TP в том числе потому, что представителей этого класса больше;
- для класса 1 — клиент откликнулся — мы получили низкие значения TN.

Поэтому значения precision и recall низкие. Модель даёт неудовлетворительные результаты, так как находит мало клиентов, которые откликнутся на предложение.



Обучите логистическую регрессию с параметрами по умолчанию и посмотрите на метрики.

Везде дальше при оценке метрик надо выводить confusion_matrix, precision и recall.

In [51]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(max_iter=100)
log_reg.fit(Xtrain, ytrain)

ypred = log_reg.predict(Xtest)

In [52]:

conf_matrix = confusion_matrix(ytest, ypred)
precision = precision_score(ytest, ypred)
recall = recall_score(ytest, ypred)

print("Матрица ошибок:")
print(conf_matrix)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")

Матрица ошибок:
[[4004    4]
 [ 559    0]]
Precision: 0.00
Recall: 0.00


Наша цель — найти как можно больше клиентов, которые откликнутся на предложение. А модель таких не находит. 

Мы помним, что метод predict_proba у логистической регрессии предсказывает математические (то есть корректные) вероятности классов. Предскажите вероятности классов с помощью обученной логистической регрессии на тестовых данных и выведите вероятности положительного класса для первых десяти объектов. 

Глядя на полученные вероятности, попробуйте объяснить, почему вы получили именно такую матрицу ошибок и такие значения точности с полноты.

In [53]:
proba = log_reg.predict_proba(Xtest)

positive_class_probabilities = proba[:, 1][:5]
print("Вероятности положительного класса:")
print(positive_class_probabilities)

Вероятности положительного класса:
[0.07132933 0.11530903 0.2918821  0.21909758 0.13873432]


Модель относит почти все объекты к 1 классу в силу низких вероятностей

Давайте уточним цель. Пусть нам нужно найти как можно больше клиентов, которые откликнутся на предложение, то есть максимизировать полноту (recall). 

При этом хочется, чтобы точность модели (precision) не была очень низкой. Заказчик посмотрел на результаты работы KNN и потребовал, чтобы точность была не ниже, чем у KNN: $precision \geq 0.13$.

Давайте будем изменять порог для перевода вероятности в классы так, чтобы:
*   максимизировать значение recall
*   при условии, что $precision \geq 0.13$.

Если мы будем подбирать порог по тестовой выборке, то, по сути, обучимся на ней и, значит, переобучимся. Это плохо. 

Поэтому предскажите вероятности на валидационной выборке и подберите порог по ней (Xval, yval), а затем посмотрите, какое качество для найденного порога вы получите на тестовых данных.

In [54]:
probs_val = log_reg.predict_proba(Xval)[:, 1]

max_recall = -1
thr = -1
prec = -1

for threshold in np.arange(0.05, 1, 0.001):
    y_val_pred = (probs_val >= threshold).astype(int)
    
    precision = precision_score(yval, y_val_pred)
    recall = recall_score(yval, y_val_pred)

    if precision >= 0.13 and recall > max_recall:
        max_recall = recall
        best_threshold = threshold
        best_precision = precision

print(f"Лучший порог: {best_threshold:.3f}, Максимальная полнота (Recall): {max_recall:.3f}, Точность (Precision): {best_precision:.3f}")

Лучший порог: 0.073, Максимальная полнота (Recall): 0.890, Точность (Precision): 0.130


In [55]:
probs_test = log_reg.predict_proba(Xtest)[:, 1]
y_test_pred = (probs_test >= best_threshold).astype(int)

# Вычисляем метрики на тестовых данных
test_precision = precision_score(ytest, y_test_pred)
test_recall = recall_score(ytest, y_test_pred)
conf_matrix = confusion_matrix(ytest, y_test_pred)

# Выводим результаты
print("\nМетрики на тестовых данных:")
print("Матрица ошибок:")
print(conf_matrix)
print(f"Precision: {test_precision:.2f}")
print(f"Recall: {test_recall:.2f}")


Метрики на тестовых данных:
Матрица ошибок:
[[1061 2947]
 [  88  471]]
Precision: 0.14
Recall: 0.84


Сделайте вывод. Смогли ли мы с помощью подбора порога добиться большего значения recall, чем у KNN? 

да

А ещё, чтобы улучшить качество предсказания, можно подбирать значение гиперпараметра C у логистической регрессии. Для каждого значения C придётся подбирать свой порог, поэтому:  

1. Обучите для значений C из диапазона [0.05, 0.15, 0.25, ...., 10.05] логистическую регрессию (на тренировочных данных).

2. Для каждой из обученных моделей во внутреннем цикле подберите оптимальный порог (как в предыдущем задании) — на валидационных данных.



В качестве результата выведите значение C и порога для модели, которая даёт наилучшие значения метрик (наибольший recall при ограничении на $precision \geq 0.13$).

Также напечатайте полученные метрики (матрицу ошибок, точность и полноту) для лучшей модели — на тестовых данных.

In [56]:
regs = []
recalls = []
thresholds = []
precisions = []

for C in np.arange(0.001, 10.1, 0.1):

    log_reg = LogisticRegression(max_iter=100, C=C)
    log_reg.fit(Xtrain, ytrain)

    probs_val = log_reg.predict_proba(Xval)[:, 1]
    
    max_recall = -1
    best_threshold = -1
    best_precision = -1
    
    for threshold in np.arange(0.05, 1, 0.001):
        y_val_pred = (probs_val >= threshold).astype(int)
        
        precision = precision_score(yval, y_val_pred)
        recall = recall_score(yval, y_val_pred)

        if precision >= 0.13 and recall > max_recall:
            max_recall = recall
            best_threshold = threshold
            best_precision = precision
            
    recalls.append(max_recall)
    precisions.append(best_precision)
    thresholds.append(best_threshold)
    regs.append(C)

In [57]:
best_index = np.argmax(recalls) 
best_C = regs[best_index]
best_threshold = thresholds[best_index]
best_precision = precisions[best_index]
best_recall = recalls[best_index]

print(f"Лучший C: {best_C:.3f}, Лучший порог: {best_threshold:.3f}, "
      f"Precision: {best_precision:.3f}, Recall: {best_recall:.3f}")

Лучший C: 9.501, Лучший порог: 0.069, Precision: 0.130, Recall: 0.926


In [58]:
best_model = LogisticRegression(max_iter=100, C=best_C)
best_model.fit(Xtrain, ytrain)

probs_test = best_model.predict_proba(Xtest)[:, 1]
y_test_pred = (probs_test >= best_threshold).astype(int)

test_precision = precision_score(ytest, y_test_pred)
test_recall = recall_score(ytest, y_test_pred)
conf_matrix = confusion_matrix(ytest, y_test_pred)

print("\nМетрики на тестовых данных:")
print("Матрица ошибок:")
print(conf_matrix)
print(f"Precision: {test_precision:.2f}")
print(f"Recall: {test_recall:.2f}")


Метрики на тестовых данных:
Матрица ошибок:
[[ 918 3090]
 [  70  489]]
Precision: 0.14
Recall: 0.87


Влияет ли изменение гиперпараметра C на качество модели (и, соответственно, метрики) в этой задаче?

влияет

Ответьте развёрнуто на следующие вопросы:

* Удалось ли при помощи логистической регрессии и подбора порога превзойти качество метода ближайших соседей в этой задаче? 

* Смогли ли мы при помощи этой модели получить высокий recall при ограничениях, поставленных заказчиком?


удалось и смог

а что тут расписывать собственно?